<a href="https://colab.research.google.com/github/shirleyrutgers/DataVis/blob/main/database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Database
Business Application and LQ Private Industry

Step 1. Download file from online and save in colab

Data

Business applications
https://docs.google.com/spreadsheets/d/e/2PACX-1vRl9ztuLbIaUzAFe1YgGB7IRhZL4P5W3dL3XeSCna8RXFdJAfIya78QR2otbTWE4w/pub?gid=13513596&single=true&output=csv

Industry Location Quotient
https://docs.google.com/spreadsheets/d/e/2PACX-1vRxQsQ5FEDOhsbuD4U76nKN8Ky_dZX6AfLQMqtqdJHHlz3guHAKUBzYb_cBfzEkaw/pub?gid=1931554944&single=true&output=csv


# Set Up

In [82]:
#---------------------------SETUP----------------------------------
#get useful libraries
import time, os, sys, re #basics
import zipfile, json, datetime, string   #string for annotating points in scatter
import numpy as np #basic math
from statistics import * #stats

import matplotlib.pyplot as plt #import pylab as plt #apparently discouraged now:
 #https://stackoverflow.com/questions/11469336/what-is-the-difference-between-pylab-and-pyplot
 #https://www.tutorialspoint.com/matplotlib/matplotlib_pylab_module.htm

import pandas as pd
import pandas_datareader as pdr
from pandas_datareader import wb
from pandas.io.formats.style import Styler
#s4 = Styler(df4, uuid_len=0, cell_ids=False)

import urllib  #weird, guess need to have os and pandas imported for this to work  %TODO/LATER ditch it, its weird anyway, just use wget/curl

from google.colab import files

#import webbrowser

import seaborn as sns

from google.colab import data_table
data_table.enable_dataframe_formatter() #this enables spreadsheet view upon calling dataframe (without() )

#tricks how to extend notebook functionality
#https://coderzcolumn.com/tutorials/python/list-of-useful-magic-commands-in-jupyter-notebook-lab
#will display all output not just last command
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

###magics: https://ipython.readthedocs.io/en/stable/interactive/magics.html
#most essential setup for vis: it does affect vis! careful!! stick with inline, maybe notebook; others mostly for non-notebook, eg spyder environ
#https://jakevdp.github.io/PythonDataScienceHandbook/04.00-introduction-to-matplotlib.html recomends *inline*!
#show current one:
#%matplotlib
#%matplotlib --list
#interactive plots:
#%matplotlib notebook
#static images of your plot:
%matplotlib inline
#this one and other magics (btw default is probably agg)
#%matplotlib nbagg
##https://www.marktechpost.com/2023/10/20/6-magic-commands-for-jupyter-notebooks-in-python-data-science/
#%%latex
#%ai
#%run
#%writefile
#%history -n

###themes/styles: https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html
#https://jakevdp.github.io/PythonDataScienceHandbook/04.11-settings-and-stylesheets.html
#https://matplotlib.org/stable/tutorials/introductory/customizing.html
#here more about art and style than under the hood functionality as with magics, explore and experiment
#many may find 'default' or seaborn ones more pleasing; my fav 'classic' is back from 90s ;)
#plt.style.available #list available styles :) may install more
plt.style.use('default') # more delicate subtle than classic
#plt.style.use('classic')  #  'seaborn-whitegrid' 'seaborn-white' 'seaborn-poster'
# btw: magics v theme/style sequence matters, eg if i specify classic style before inline magic, i wouldnt get grey bounding box im getting

#sometimes have to install library which you get from https://pypi.org/
#!pip install geopandas

#Database 1. Business Application

In [83]:
# Read Database 1, Business Application, from URL CSV, from row 3
business = pd.read_csv(
    'https://docs.google.com/spreadsheets/d/e/2PACX-1vRl9ztuLbIaUzAFe1YgGB7IRhZL4P5W3dL3XeSCna8RXFdJAfIya78QR2otbTWE4w/pub?gid=13513596&single=true&output=csv',
    skiprows=2
)

# Review
business.head()

,State,County,County Code,fipstate,fipscty,BA2005,BA2006,BA2007,BA2008,BA2009,...,BA2014,BA2015,BA2016,BA2017,BA2018,BA2019,BA2020,BA2021,BA2022,BA2023
0,AL,Autauga County,1001,1,1,335,348,348,367,288,...,290,291,294,313,384,394,559,776,707,703
1,AL,Baldwin County,1003,1,3,1986,2101,1975,1721,1505,...,1575,1735,1858,2114,2104,2317,2697,3407,3694,3855
2,AL,Barbour County,1005,1,5,138,179,156,139,133,...,120,129,122,114,113,135,230,334,266,421
3,AL,Bibb County,1007,1,7,118,145,117,98,98,...,101,78,86,79,105,86,165,199,158,173
4,AL,Blount County,1009,1,9,315,323,297,306,247,...,228,218,254,281,299,312,344,473,428,513


In [84]:
# Delete unnecessary columns from business
cols_to_drop = [
    'fipstate', 'fipscty',        # IDs duplicados
    'BA2005', 'BA2006', 'BA2007', 'BA2008', 'BA2009',
    'BA2010', 'BA2011', 'BA2012', 'BA2013', 'BA2014',
    'BA2015', 'BA2016', 'BA2017', 'BA2018', 'BA2019',
    'BA2020', 'BA2021', 'BA2022'  # All years before 2023
]

# Apply the drop
business = business.drop(columns=cols_to_drop)

# Review the result
business.head()

,State,County,County Code,BA2023
0,AL,Autauga County,1001,703
1,AL,Baldwin County,1003,3855
2,AL,Barbour County,1005,421
3,AL,Bibb County,1007,173
4,AL,Blount County,1009,513


In [85]:
#Before merge, review the data type
print(business.dtypes)

State          object
County         object
County Code     int64
BA2023         object
dtype: object


In [86]:
# First: make County Code a string
business['County Code'] = business['County Code'].astype(str)

# Second: Fill County Code with zeros to ensure 5 digits
business['County Code'] = business['County Code'].str.zfill(5)

# Database 2. Industry

In [87]:
#Read Database 2> Industry Data from URL CSV
industry=pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRxQsQ5FEDOhsbuD4U76nKN8Ky_dZX6AfLQMqtqdJHHlz3guHAKUBzYb_cBfzEkaw/pub?gid=1931554944&single=true&output=csv')
# Review
industry.head()

<ipython-input-87-bf5e9b1227ce>:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  industry=pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRxQsQ5FEDOhsbuD4U76nKN8Ky_dZX6AfLQMqtqdJHHlz3guHAKUBzYb_cBfzEkaw/pub?gid=1931554944&single=true&output=csv')


,Area\nCode,St,Cnty,Own,NAICS,Year,Qtr,Area Type,St Name,Area,Ownership,Industry,Annual Average Status Code,Annual Average Establishment Count,Annual Average Employment,Annual Total Wages,Annual Average Weekly Wage,Annual Average Pay,Employment Location Quotient Relative to U.S.,Total Wage Location Quotient Relative to U.S.
0,US000,US,0.0,0,10,2023,A,Nation,NaN,U.S. TOTAL,Total Covered,"10 Total, all industries",NaN,"11,866,306","153,140,899","11,081,267,615,470","1,392","72,360",1.0,1.0
1,US000,US,0.0,1,10,2023,A,Nation,NaN,U.S. TOTAL,Federal Government,"10 Total, all industries",NaN,"61,310","2,929,938","286,764,744,238","1,882","97,874",1.0,1.0
2,US000,US,0.0,2,10,2023,A,Nation,NaN,U.S. TOTAL,State Government,"10 Total, all industries",NaN,"72,215","4,658,153","356,315,881,354","1,471","76,493",1.0,1.0
3,US000,US,0.0,3,10,2023,A,Nation,NaN,U.S. TOTAL,Local Government,"10 Total, all industries",NaN,"171,844","14,263,127","905,493,894,970","1,221","63,485",1.0,1.0
4,US000,US,0.0,5,10,2023,A,Nation,NaN,U.S. TOTAL,Private,"10 Total, all industries",NaN,"11,560,938","131,289,681","9,532,693,094,908","1,396","72,608",1.0,1.0


In [88]:
#Cleanning the database> industry
#Rename the column Area
industry = industry.rename(columns={"Area": "County"})
#Rename the column Area Code
industry = industry.rename(columns={'Area\nCode': 'County Code'})

In [89]:
# Filter> County, industry Private Ownership, and Total Industry, delete "Unknow Or Undefined"
priv= industry[
    (industry["Area Type"] == "County") &
    (industry["Ownership"] == "Private") &
    (industry["Industry"] == "10 Total, all industries") &
    (~industry["County"].str.contains("Unknown Or Undefined", na=False))
]

# Display the filtered results
priv.head()

,County Code,St,Cnty,Own,NAICS,Year,Qtr,Area Type,St Name,County,Ownership,Industry,Annual Average Status Code,Annual Average Establishment Count,Annual Average Employment,Annual Total Wages,Annual Average Weekly Wage,Annual Average Pay,Employment Location Quotient Relative to U.S.,Total Wage Location Quotient Relative to U.S.
40,01001,01,1.0,5,10,2023,A,County,Alabama,"Autauga County, Alabama",Private,"10 Total, all industries",NaN,"1,011","9,330","454,329,752",936,"48,696",0.92,0.89
57,01003,01,3.0,5,10,2023,A,County,Alabama,"Baldwin County, Alabama",Private,"10 Total, all industries",NaN,"8,137","71,867","3,473,350,164",929,"48,330",1.01,1.00
75,01005,01,5.0,5,10,2023,A,County,Alabama,"Barbour County, Alabama",Private,"10 Total, all industries",NaN,533,"6,227","264,209,398",816,"42,433",0.94,0.91
92,01007,01,7.0,5,10,2023,A,County,Alabama,"Bibb County, Alabama",Private,"10 Total, all industries",NaN,380,"3,529","187,928,363","1,024","53,248",0.85,0.88
109,01009,01,9.0,5,10,2023,A,County,Alabama,"Blount County, Alabama",Private,"10 Total, all industries",NaN,867,"6,734","293,597,353",839,"43,602",0.89,0.88


In [90]:
#Before merge, review the data type
print(priv.dtypes)

County Code                                       object
St                                                object
Cnty                                             float64
Own                                                int64
NAICS                                              int64
Year                                               int64
Qtr                                               object
Area Type                                         object
St Name                                           object
County                                            object
Ownership                                         object
Industry                                          object
Annual Average Status Code                        object
Annual Average Establishment Count                object
Annual Average Employment                         object
Annual Total Wages                                object
Annual Average Weekly Wage                        object
Annual Average Pay             

# Merge

In [96]:
print(business.dtypes)
print(priv.dtypes)

State          object
County         object
County Code    object
BA2023         object
dtype: object
County Code                                       object
St                                                object
Cnty                                             float64
Own                                                int64
NAICS                                              int64
Year                                               int64
Qtr                                               object
Area Type                                         object
St Name                                           object
County                                            object
Ownership                                         object
Industry                                          object
Annual Average Status Code                        object
Annual Average Establishment Count                object
Annual Average Employment                         object
Annual Total Wages                         

In [97]:
#TEST
# New business dataset: only County Code and BA2023
business_filtered = business[['County Code', 'BA2023']]

# New priv dataset: only County Code and Employment Location Quotient Relative to U.S.
priv_filtered = priv[['County Code', 'Employment Location Quotient Relative to U.S.']]

# Review the first few rows
business_filtered.head()
priv_filtered.head()


,County Code,BA2023
0,01001,703
1,01003,3855
2,01005,421
3,01007,173
4,01009,513


,County Code,Employment Location Quotient Relative to U.S.
40,01001,0.92
57,01003,1.01
75,01005,0.94
92,01007,0.85
109,01009,0.89


In [98]:
# Merge business and priv where County Code matches
#outer=keep all
merged = pd.merge(business, priv, how='outer', on='County Code', indicator=True)
# Review the first rows
merged

,State,County_x,County Code,BA2023,St,Cnty,Own,NAICS,Year,Qtr,...,Industry,Annual Average Status Code,Annual Average Establishment Count,Annual Average Employment,Annual Total Wages,Annual Average Weekly Wage,Annual Average Pay,Employment Location Quotient Relative to U.S.,Total Wage Location Quotient Relative to U.S.,_merge
0,AL,Autauga County,01001,703,01,1.0,5.0,10.0,2023.0,A,...,"10 Total, all industries",NaN,"1,011","9,330","454,329,752",936,"48,696",0.92,0.89,both
1,AL,Baldwin County,01003,3855,01,3.0,5.0,10.0,2023.0,A,...,"10 Total, all industries",NaN,"8,137","71,867","3,473,350,164",929,"48,330",1.01,1.00,both
2,AL,Barbour County,01005,421,01,5.0,5.0,10.0,2023.0,A,...,"10 Total, all industries",NaN,533,"6,227","264,209,398",816,"42,433",0.94,0.91,both
3,AL,Bibb County,01007,173,01,7.0,5.0,10.0,2023.0,A,...,"10 Total, all industries",NaN,380,"3,529","187,928,363","1,024","53,248",0.85,0.88,both
4,AL,Blount County,01009,513,01,9.0,5.0,10.0,2023.0,A,...,"10 Total, all industries",NaN,867,"6,734","293,597,353",839,"43,602",0.89,0.88,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3152,WY,Sweetwater County,56037,839,56.0,37.0,5.0,10.0,2023.0,A,...,"10 Total, all industries",NaN,"1,588","16,760","1,178,570,755","1,352","70,321",0.94,0.97,both
3153,WY,Teton County,56039,1374,56.0,39.0,5.0,10.0,2023.0,A,...,"10 Total, all industries",NaN,"2,916","20,680","1,590,595,650","1,479","76,914",1.03,1.03,both
3154,WY,Uinta County,56041,290,56.0,41.0,5.0,10.0,2023.0,A,...,"10 Total, all industries",NaN,803,"6,115","304,737,470",958,"49,836",0.87,0.89,both
3155,WY,Washakie County,56043,112,56.0,43.0,5.0,10.0,2023.0,A,...,"10 Total, all industries",NaN,404,"2,818","136,283,514",930,"48,356",0.91,0.91,both


In [99]:
#Results of merge
#Summary of merge results
merge_summary = merged['_merge'].value_counts()

print(merge_summary)


_merge
both          3142
left_only       14
right_only       1
Name: count, dtype: int64


In [100]:
# Count unique County Codes
unique_county_codes = merged['County Code'].nunique()

print(f"Unique County Codes: {unique_county_codes}")


Unique County Codes: 3157


In [101]:
# Total rows after merge
total_rows = merged.shape[0]

print(f"Total Rows in merged: {total_rows}")

Total Rows in merged: 3157


In [102]:
# Summary of match status
print(merged['_merge'].value_counts())

# How many unique County Codes
print(f"Unique County Codes: {merged['County Code'].nunique()}")

# Total rows
print(f"Total Rows: {merged.shape[0]}")


_merge
both          3142
left_only       14
right_only       1
Name: count, dtype: int64
Unique County Codes: 3157
Total Rows: 3157


In [103]:
# Filter for left_only (only in business, no match in priv)
left_only = merged[merged['_merge'] == 'left_only']

# Filter for right_only (only in priv, no match in business)
right_only = merged[merged['_merge'] == 'right_only']

# Review
print("Left Only (business but no priv):")
print(left_only)

print("\nRight Only (priv but no business):")
print(right_only)


Left Only (business but no priv):
     State                                        County_x County Code BA2023  \
94      AK                      Valdez-Cordova Census Area       02261          
95      AK                        Wade Hampton Census Area       02270          
319     CT                         Capitol Planning Region       09110  11967   
320     CT              Greater Bridgeport Planning Region       09120   5064   
321     CT  Lower Connecticut River Valley Planning Region       09130   1858   
322     CT                Naugatuck Valley Planning Region       09140   5451   
323     CT        Northeastern Connecticut Planning Region       09150    819   
324     CT                 Northwest Hills Planning Region       09160   1188   
325     CT       South Central Connecticut Planning Region       09170   7390   
326     CT        Southeastern Connecticut Planning Region       09180   2577   
327     CT             Western Connecticut Planning Region       09190  116

In [104]:
# Filter for left_only (only in business, no match in priv)
left_only = merged[merged['_merge'] == 'left_only']

# Filter for right_only (only in priv, no match in business)
right_only = merged[merged['_merge'] == 'right_only']

# Display as nice tables
from IPython.display import display

print("Left Only (business but no priv):")
display(left_only)

print("\nRight Only (priv but no business):")
display(right_only)


Left Only (business but no priv):


,State,County_x,County Code,BA2023,St,Cnty,Own,NAICS,Year,Qtr,...,Industry,Annual Average Status Code,Annual Average Establishment Count,Annual Average Employment,Annual Total Wages,Annual Average Weekly Wage,Annual Average Pay,Employment Location Quotient Relative to U.S.,Total Wage Location Quotient Relative to U.S.,_merge
94,AK,Valdez-Cordova Census Area,02261,,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
95,AK,Wade Hampton Census Area,02270,,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
319,CT,Capitol Planning Region,09110,11967,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
320,CT,Greater Bridgeport Planning Region,09120,5064,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
321,CT,Lower Connecticut River Valley Planning Region,09130,1858,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
322,CT,Naugatuck Valley Planning Region,09140,5451,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
323,CT,Northeastern Connecticut Planning Region,09150,819,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
324,CT,Northwest Hills Planning Region,09160,1188,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
325,CT,South Central Connecticut Planning Region,09170,7390,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
326,CT,Southeastern Connecticut Planning Region,09180,2577,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only



Right Only (priv but no business):


,State,County_x,County Code,BA2023,St,Cnty,Own,NAICS,Year,Qtr,...,Industry,Annual Average Status Code,Annual Average Establishment Count,Annual Average Employment,Annual Total Wages,Annual Average Weekly Wage,Annual Average Pay,Employment Location Quotient Relative to U.S.,Total Wage Location Quotient Relative to U.S.,_merge
332,NaN,NaN,11999,NaN,11,999.0,5.0,10.0,2023.0,A,...,"10 Total, all industries",NaN,14,54,"11,669,668","4,188","217,785",1.17,1.16,right_only
